# Split SPP into Sub-Areas
SPP: [Sothwest Power Tool](https://www.spp.org/)

We have contacted SPP people through their [internal request management system](https://spprms.issuetrak.com/Login.asp) to obtain a list of counties for owners reporting to SPP.

In [1]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict
from powersimdata.input.grid import Grid

## 1. Create Bus to Balancing Authority area Mapping via County
Load the list of counties for *owners* reporting to SPP. 

In [2]:
county_list_owner = pd.read_excel('spp_counties_owners.xlsx', index_col=0)
county_list_owner.tail()

,STATE,COUNTY,SQUARE_MIL,OWNER,SPP
OBJECTID,,,,,
4505,TX,Foard County,707.471,SPS,Y
1332,ND,Renville County,893.326,UMZ,Y
3835,OK,Alfalfa County,883.528,WFEC,Y
2271,SD,Charles Mix County,1150.352,UMZ,Y
2543,IA,Harrison County,696.375,UMZ,Y


Manually map legacy BA to owners.

In [3]:
legacy_ba_to_owners = {
    'CSWS': {'AEP', 'AECC', 'SPA'}, # AECC and SPA are approximately merged into CSWS
    'EDE': {'EDE'},
    'MPS': {'GMO'},
    'GRDA': {'GRDA'},
    'KACY': {'KACY'},
    'KCPL_INDN': {'KCPL'},
    'LES': {'LES'},
    'NPPD': {'NPPD'},
    'OKGE': {'OGE'},
    'OPPD': {'OPPD'},
    'SPRM': {'SPRM'},
    'SPS': {'SPS'},
    'SECI': {'SUNC', 'MIDW', 'MKEC'}, # MIDW and MKEC are approximately merged into SECI
    'WAUE': {'UMZ'},
    'WR': {'WESTAR'},
    "WFEC": {'WFEC'},
}
# legacy BA INDN has hourly load profile, but the territory definition is missing. We merge the load of INDN into KCPL at this point.

Map owner to county

In [4]:
owner_county_list = defaultdict(set)
for index, row in county_list_owner.iterrows():
    owner_county_list[row['OWNER']].add(''.join(row['COUNTY'].split()[:-1]) + '__' + row['STATE'])

Map legacy BA to county.

In [5]:
legacy_ba_county_list = defaultdict(set)
for legacy_ba,owners in legacy_ba_to_owners.items():
    for owner in owners:
        legacy_ba_county_list[legacy_ba] |= owner_county_list[owner]

Load the bus to BA mapping for the Eastern interconnect previously produced [here](../../eastern_demand_v5_demo/bus_ba_map.ipynb) and only consider entries for SPP.

In [6]:
bus_ba_map = pd.read_csv(os.path.join("..", "..", "eastern_demand_v5_demo/bus_ba_map.csv"), index_col=0)
bus_ba_map_spp = bus_ba_map[bus_ba_map['BA'] == 'SWPP'].copy()

Assign a legacy BA to the buses located in SPP.

In [7]:
for index,row in bus_ba_map_spp.iterrows():
    for legacy_ba, county_list in legacy_ba_county_list.items():
        if row['County'] in county_list:
            bus_ba_map_spp.loc[index, 'BA'] = legacy_ba

Some buses have not been mapped to a legacy BA. Those are handled manually.

In [8]:
bus_no_match = bus_ba_map_spp[bus_ba_map_spp['BA'] == 'SWPP']
bus_no_match

,Pd,lat,lon,County,BA,zone_name
bus_id,,,,,,
32119,34.02,36.625239,-82.151913,Bristol City__VA,SWPP,Tennessee
45795,1.21,38.006546,-90.177534,Ste. Genevieve__MO,SWPP,Illinois Downstate
45958,2.09,37.885677,-90.001297,Ste. Genevieve__MO,SWPP,Illinois Downstate
45959,3.91,37.827114,-89.941909,Perry__MO,SWPP,Illinois Downstate
49501,2.98,48.608566,-97.198213,Pembina__ND,SWPP,Minnesota Northern
...,...,...,...,...,...,...
69834,0.46,47.195438,-107.690116,Garfield__MT,SWPP,Montana Eastern
69863,2.58,45.461193,-106.199382,Powder River__MT,SWPP,Montana Eastern
69864,1.33,45.368461,-106.180413,Powder River__MT,SWPP,Montana Eastern


In [9]:
# Manual fix the buses that locate in the counties not in the legacy county list
for index,row in bus_no_match.iterrows():
    if row['County'][-2:] in {'MT', 'ND', 'SD', 'WY'}:
        bus_ba_map_spp.loc[index, 'BA'] = 'WAUE'
    if row['County'][-2:] in {'NE'}:
        bus_ba_map_spp.loc[index, 'BA'] = 'NPPD'
    if row['County'][-2:] in {'NM'}:
        bus_ba_map_spp.loc[index, 'BA'] = 'SPS'
    if row['County'][-2:] in {'OK', 'LA', 'TX', 'AR'}:
        bus_ba_map_spp.loc[index, 'BA'] = 'CSWS'
    if row['zone_name'] in {'Missouri West', 'Kansas','Iowa'}:
        bus_ba_map_spp.loc[index, 'BA'] = 'KCPL_INDN'
    if row['County'][-2:] in {'VA'}:
        bus_ba_map_spp.loc[index, 'BA'] = 'PJM'
# The rest of the buses belong to another ISO, AECI, which previously merged into MISO
bus_ba_map_spp.replace('SWPP', 'AECI', inplace=True)

In [10]:
bus_no_match = bus_ba_map_spp[bus_ba_map_spp['BA'] == 'SWPP']
bus_no_match

,Pd,lat,lon,County,BA,zone_name
bus_id,,,,,,


Write data frame mapping buses located in SPP to legacy BA in a file.

In [11]:
bus_ba_map_spp.to_csv('bus_ba_map_spp.csv')

## 2. Sanity check
Plot on the counties of a sub area on the map.

In [12]:
from bokeh.io import show,output_notebook,output_file,export
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    CustomJS,
    Slider,
    CheckboxGroup,
    Select,
    Div,
    TapTool,
    ColorBar,
    LinearColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.layouts import (widgetbox, row, gridplot)
from bokeh.sampledata.us_counties import data as us_counties
from bokeh.sampledata.us_states import data as us_states
from bokeh import events
import matplotlib.pyplot as plt
from bokeh.colors.groups import yellow
import bokeh.colors
import matplotlib.cm as cm
import matplotlib

In [13]:
state_xs = [us_states[state]["lons"] for state in us_states if state not in {'HI', 'AK'}]
state_ys = [us_states[state]["lats"] for state in us_states if state not in {'HI', 'AK'}]

In [14]:
area_label = {}
cnt = 0
for area in legacy_ba_to_owners:
    cnt += 1
    area_label[area] = cnt
legacy_ba_county_list_plot = {}
for index,row in bus_ba_map_spp.iterrows():
    if row['BA'] in area_label:
        legacy_ba_county_list_plot[
            row['County'][:-4] + '_' + row['County'][-2:].lower()] = (row['BA'], area_label[row['BA']])

In [15]:
counties_xs = []
counties_ys = []
counties_name = []
area_color = []
for code in us_counties:
    key = str(us_counties[code]['name']) + '_' + str(us_counties[code]['state'])
    if key in legacy_ba_county_list_plot:
        counties_xs.append(us_counties[code]['lons'])
        counties_ys.append(us_counties[code]['lats'])
        counties_name.append(key+':'+legacy_ba_county_list_plot[key][0])
        area_color.append(bokeh.palettes.Category20[20][(legacy_ba_county_list_plot[key][1]-1)%20])
source = ColumnDataSource(data=dict(x=counties_xs,
                                    y=counties_ys,
                                    name=counties_name,
                                    color=area_color))

In [16]:
p = figure(title="BA County Check", 
           toolbar_location="left", plot_width=1300, plot_height=700,
           tools = ['tap', 'box_zoom', 'wheel_zoom', 'save', 'reset'])

p.patches(state_xs, state_ys,
          fill_alpha=0,
          line_color="gray",
          line_alpha=0.5,
          line_width=1.0)

county_render = p.patches('x', 'y',
                          source=source,
                          fill_alpha=0.5,
                          line_color="gray",
                          line_alpha=0.5,
                          line_width=0.5,
                          color='color')

hover1 = HoverTool(renderers=[county_render], name='Test')
hover1.point_policy = "follow_mouse"
hover1.tooltips = [("County", "@name")]
p.add_tools(hover1)

show(p)

In [17]:
eastern = Grid(['Eastern'])

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv


In [18]:
# Plot for buses assigned to a specific BA area
source = eastern.bus.loc[list(bus_ba_map_spp[bus_ba_map_spp['BA'].isin(legacy_ba_to_owners)].index)]
source.loc[:,'area_color'] = source['zone_id'].apply(lambda x: bokeh.palettes.Category20[20][(x-1)%20])

p = figure(title="", 
           toolbar_location="left",
           plot_width=1300, 
           plot_height=700)

p.patches(state_xs, state_ys, 
          fill_alpha=0,
          line_color="gray",
          line_alpha=0.5,
          line_width=1.0)
p.patches(counties_xs, counties_ys, 
          fill_alpha=0,
          line_color="gray",
          line_alpha=0.5, 
          line_width=0.5)

p.scatter(source=source, x='lon', y='lat', size=2, marker='o', color='area_color')

show(p)